In [1]:
import os
import sys
import glob
import numpy as np
import torch
from torchvision import transforms, datasets
from PIL import Image

#Import image loaders
os.chdir("../../src/")
from load_data import preview_images, load_images

**Load Data**

In [2]:
#Import and preview image data
train_path= "../Data/train/"
horse_train= load_images(train_path + "horses/")
human_train= load_images(train_path + "humans/")

val_path= "../Data/validation/"
horse_test= load_images(val_path + "horses/")
human_test= load_images(val_path + "humans/")

In [3]:
#Sanity Check
print("Training Horse Data: %s Test Horse Data: %s" % (horse_train.shape, horse_test.shape))
print("Training Human Data: %s Test Human Data: %s" % (human_train.shape, human_test.shape))

Training Horse Data: (500, 300, 300, 4) Test Horse Data: (128, 300, 300, 4)
Training Human Data: (527, 300, 300, 4) Test Human Data: (128, 300, 300, 4)


**Preprocess Data**

In [4]:
#Assign labels: 1= human, 0 = horse
y_train_human= np.full((len(human_train),1), 1)
y_train_horse= np.full((len(horse_train),1), 0)

In [5]:
#Concatenate Training data
X= np.concatenate((horse_train, human_train), axis=0)
y= np.concatenate((y_train_horse, y_train_human), axis= 0)

In [6]:
#Zero-center data
training_mean= X.mean()
training_std= X.std()
X= (X - training_mean)/training_std

In [7]:
#Shuffle
from sklearn.utils import shuffle
X_train, y_train= shuffle(X, y)

#Sanity check
print("Training data shape: %s \n Training label shape: %s" % (X_train.shape, y_train.shape))

Training data shape: (1027, 300, 300, 4) 
 Training label shape: (1027, 1)


**Modeling**

In [8]:
#use base model from early computer vision
from convnet import BaseNeuralNetwork

base_model= BaseNeuralNetwork()
print(base_model)

BaseNeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=90000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=2, bias=True)
  )
)


In [10]:
X= torch.from_numpy(X_train)

In [11]:
logits= base_model(X)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1027x360000 and 90000x512)

In [ ]:
pred_probs= torch.nn.Softmax(logits)

In [ ]:
pred_probs[:10]

In [ ]:
#PyTorch CNN

In [ ]:
do not run below this line

**Deploy**

In [ ]:
y_test_human= np.full((len(human_test),1), 1)
y_test_horse= np.full((len(horse_test),1), 0)

#Concatenate validation set
X= np.concatenate((horse_test, human_test), axis=0)
y= np.concatenate((y_test_horse, y_test_human), axis= 0)

#Zero-Center data
X= (X- training_mean)/training_std

#Shuffle
X_test, y_test= shuffle(X, y)

#Sanity check
print("Test data shape: %s \n Training label shape: %s" % (X_test.shape, y_test.shape))